# Graph-Based Feature Engineering for Flight Delay Prediction

**Author:** Daniel Costa  
**Project:** Flight Delay Prediction (W261 Final Project)  
**Dataset:** US Domestic Flights (2015-2019)

---

## Overview

This notebook engineers **graph-based features** from the flight network to capture airport connectivity and route characteristics that influence flight delays. The US flight network is modeled as a directed graph where airports are nodes and flight routes are edges.

### Feature Categories

| Category | Features | Description |
|----------|----------|-------------|
| **Centrality Metrics** | PageRank, Betweenness, Closeness | Measures of airport importance in the network |
| **Degree Metrics** | In/Out Degree (weighted & unweighted) | Airport connectivity and traffic volume |
| **Infrastructure** | Number of Runways | Physical capacity constraints |
| **Route Features** | Avg Daily Flights, Avg Delay, Hourly Traffic | Route-specific patterns |
| **Historical Delays** | Avg Origin/Destination Delay | Airport-level delay tendencies |

### Network Representation

```
                    ┌─────────┐
        ┌──────────►│   ATL   │◄──────────┐
        │           └────┬────┘           │
        │                │                │
   ┌────┴────┐      ┌────▼────┐      ┌────┴────┐
   │   ORD   │◄────►│   DFW   │◄────►│   LAX   │
   └────┬────┘      └────┬────┘      └────┬────┘
        │                │                │
        └───────────►┌───▼───┐◄───────────┘
                     │  JFK  │
                     └───────┘

Nodes = Airports (with centrality, degree, runway features)
Edges = Routes (with traffic volume, delay patterns)
```

---

## Table of Contents

1. [Setup & Configuration](#1-setup-configuration)
2. [Data Loading & Preprocessing](#2-data-loading-preprocessing)
3. [Utility Functions](#3-utility-functions)
4. [Vertex (Airport) Features](#4-vertex-airport-features)
   - [4.1 PageRank](#41-pagerank)
   - [4.2 Degree Metrics](#42-degree-metrics)
   - [4.3 Airport Infrastructure](#43-airport-infrastructure)
   - [4.4 Centrality Metrics](#44-centrality-metrics)
   - [4.5 Weighted Betweenness Centrality](#45-weighted-betweenness-centrality)
   - [4.6 Historical Delay Features](#46-historical-delay-features)
5. [Edge (Route) Features](#5-edge-route-features)
6. [Feature Pipeline](#6-feature-pipeline)
7. [Execution](#7-execution)
8. [Summary](#8-summary)

## 1. Setup & Configuration

## 2. Data Loading & Preprocessing

In [0]:


def load_airport_coordinates():
    """
    Load airport coordinates from CSV and extract latitude/longitude.
    
    Returns:
        DataFrame: Airport IATA codes with lat/long coordinates
        dict: Mapping of IATA code to {lat, long} for fast lookups
    """
    df_airports = (
        spark.read.csv(AIRPORT_CODES_PATH, header=True, inferSchema=True)
        .withColumn("long", F.split(F.col("coordinates"), ",")[0].cast("float"))
        .withColumn("lat", F.split(F.col("coordinates"), ",")[1].cast("float"))
        .select("iata_code", "long", "lat")
        .dropna()
    )
    
    # Create lookup dictionary for fast access
    airport_info = {
        row["iata_code"]: {"long": row["long"], "lat": row["lat"]}
        for row in df_airports.collect()
    }
    
    return df_airports, airport_info


def transform_df(df):
    """
    Apply standard transformations to flight DataFrame.
    
    Args:
        df: Raw flight DataFrame
        
    Returns:
        DataFrame with parsed date and hour columns
    """
    return (
        df
        .withColumn("FL_DATE", F.to_date("FL_DATE", "yyyy-MM-dd"))
        .withColumn("hour", (F.col("CRS_DEP_TIME").cast("int") / F.lit(100)).cast("int"))
    )


def ingest_data(dataset_path):
    """
    Load train, validation, and test datasets from parquet files.
    
    Args:
        dataset_path: Base path to dataset directory
        
    Returns:
        Tuple of (train_df, validation_df, test_df)
    """
    train_df = transform_df(spark.read.parquet(f"{dataset_path}/train.parquet"))
    validation_df = transform_df(spark.read.parquet(f"{dataset_path}/validation.parquet"))
    test_df = transform_df(spark.read.parquet(f"{dataset_path}/test.parquet"))
    
    return train_df, validation_df, test_df


def get_dataset_days(flight_df):
    """
    Calculate the number of days spanned by the dataset.
    
    Args:
        flight_df: Flight DataFrame with FL_DATE column
        
    Returns:
        int: Number of days in dataset
    """
    dates = flight_df.agg(
        F.min("FL_DATE").alias("min_date"),
        F.max("FL_DATE").alias("max_date")
    ).collect()[0]
    
    return (dates["max_date"] - dates["min_date"]).days + 1


# Load airport data
df_airports, airport_info = load_airport_coordinates()
print(f"Loaded {len(airport_info)} airports with coordinates")


## 3. Utility Functions

In [0]:
def haversine_np(lat1, lon1, lat2, lon2, radius=EARTH_RADIUS_M):
    """
    Calculate great-circle distance between two points using the Haversine formula.
    
    Vectorized implementation that works with NumPy arrays for efficiency.
    
    Args:
        lat1, lon1: Latitude and longitude of first point(s) in degrees
        lat2, lon2: Latitude and longitude of second point(s) in degrees
        radius: Earth's radius (default: 6,371,008.8 meters)
        
    Returns:
        Distance in kilometers
    """
    lat1 = np.asarray(lat1, dtype=np.float64)
    lon1 = np.asarray(lon1, dtype=np.float64)
    lat2 = np.asarray(lat2, dtype=np.float64)
    lon2 = np.asarray(lon2, dtype=np.float64)

    # Convert degrees to radians
    φ1, λ1 = np.radians(lat1), np.radians(lon1)
    φ2, λ2 = np.radians(lat2), np.radians(lon2)

    dφ = φ2 - φ1
    dλ = λ2 - λ1

    # Haversine formula
    s = np.sin(dφ * 0.5)
    t = np.sin(dλ * 0.5)
    a = s * s + np.cos(φ1) * np.cos(φ2) * t * t

    # Return distance in kilometers
    return (2.0 * radius) * np.arcsin(np.minimum(1.0, np.sqrt(a))) / 1000


## 4. Vertex (Airport) Features

Airport-level features capture the structural importance and operational characteristics of each airport in the flight network.

### 4.1 PageRank

PageRank measures the importance of an airport based on the number and quality of incoming flight connections.

In [0]:
def page_rank(flight_df, d=0.85, max_iter=20):
    """
    Compute PageRank for airports based on flight connections.
    
    PageRank measures the importance of an airport based on the number and
    quality of incoming flight connections. Airports with more flights from
    other important airports will have higher PageRank scores.
    
    Args:
        flight_df: Flight DataFrame with ORIGIN, DEST columns
        d: Damping factor (probability of following a link vs random jump)
        max_iter: Number of iterations for convergence
        
    Returns:
        DataFrame with ORIGIN and page_rank columns
    """
    # Build weighted edge list (weight = number of unique flights)
    edges = (
        flight_df
        .groupBy("ORIGIN", "DEST")
        .agg(F.countDistinct("flight_uid").alias("weight"))
        .withColumnRenamed("ORIGIN", "src")
        .withColumnRenamed("DEST", "dst")
    )

    # Normalize edge weights by source out-degree
    out_sums = edges.groupBy("src").agg(F.sum("weight").alias("total_w"))
    edges_norm = (
        edges.join(out_sums, on="src")
        .withColumn("norm_w", F.col("weight") / F.col("total_w"))
    )

    # Initialize all vertices with PageRank = 1.0
    vertices = edges.select("src").union(edges.select("dst")).distinct()
    ranks = vertices.withColumn("page_rank", F.lit(1.0))

    # Iterative PageRank computation
    for _ in range(max_iter):
        contribs = (
            edges_norm.join(ranks, edges_norm["src"] == ranks["src"])
            .select(
                edges_norm.dst.alias("id"),
                (F.col("page_rank") * F.col("norm_w")).alias("contrib")
            )
        )
        new_ranks = contribs.groupBy("id").agg(F.sum("contrib").alias("sum_contrib"))
        ranks = new_ranks.select(
            F.col("id").alias("src"),
            ((1 - d) + d * F.col("sum_contrib")).alias("page_rank")
        )

    return ranks.withColumnRenamed("src", "ORIGIN")


### 4.2 Degree Metrics

Degree metrics measure airport connectivity - how many unique connections and total flight volume each airport handles.

In [0]:
def out_degree(flight_df):
    """
    Calculate unweighted out-degree (number of unique destinations from each airport).
    """
    return flight_df.groupBy("ORIGIN").agg(F.countDistinct("DEST").alias("out_degree"))


def in_degree(flight_df):
    """
    Calculate unweighted in-degree (number of unique origins flying to each airport).
    """
    return (
        flight_df.groupBy("DEST")
        .agg(F.countDistinct("ORIGIN").alias("in_degree"))
        .withColumnRenamed("DEST", "ORIGIN")
    )


def weighted_in_out_degree(flight_df, dataset_n_days):
    """
    Calculate weighted in/out degree based on average daily flight volume.
    
    Unlike unweighted degree which counts unique connections, weighted degree
    captures actual traffic volume - an airport with 100 daily flights to 5
    destinations has higher weighted out-degree than one with 10 flights to 20.
    
    Args:
        flight_df: Flight DataFrame
        dataset_n_days: Number of days in dataset for normalization
        
    Returns:
        DataFrame with weighted_in_degree and weighted_out_degree columns
    """
    weighted_in = (
        flight_df.groupBy("DEST")
        .agg((F.count("*") / F.lit(dataset_n_days)).alias("weighted_in_degree"))
        .withColumnRenamed("DEST", "ORIGIN")
    )

    weighted_out = (
        flight_df.groupBy("ORIGIN")
        .agg((F.count("*") / F.lit(dataset_n_days)).alias("weighted_out_degree"))
    )

    return weighted_in.join(weighted_out, on="ORIGIN")


### 4.3 Airport Infrastructure

Physical infrastructure constraints like runway count affect airport capacity and delay potential.

In [0]:
def get_runways(flights_df):
    """
    Load airport runway counts from embedded data.
    
    Number of runways affects airport capacity and potential for delays.
    Major hubs like ORD (8 runways) can handle more simultaneous operations
    than smaller airports.
    
    Returns:
        DataFrame with ORIGIN and N_RUNWAYS columns
    """
    airport_runways = pd.read_csv(StringIO(RUNWAY_DATA))
    return spark.createDataFrame(airport_runways)


### 4.4 Centrality Metrics

Centrality metrics measure the importance of airports in the network structure.

In [0]:
# Note: airport_info lookup dictionary is created during data loading (Section 2)

In [0]:
def get_unweighted_centralities(flight_df):
    """
    Calculate betweenness and closeness centrality using NetworkX.
    
    - Betweenness Centrality: Measures how often an airport lies on the shortest
      path between other airports. High betweenness = important hub for connections.
    
    - Closeness Centrality: Measures how close an airport is to all other airports.
      Uses geographic distance as edge weights.
    
    Args:
        flight_df: Flight DataFrame with ORIGIN, DEST columns
        
    Returns:
        DataFrame with betweenness_unweighted and closeness columns
    """
    # Build directed graph with geographic distances as weights
    edges = flight_df.select("ORIGIN", "DEST").distinct().collect()
    
    G = nx.DiGraph()
    for row in edges:
        lat1, lon1 = airport_info[row["ORIGIN"]]["lat"], airport_info[row["ORIGIN"]]["long"]
        lat2, lon2 = airport_info[row["DEST"]]["lat"], airport_info[row["DEST"]]["long"]
        distance = haversine_np(lat1, lon1, lat2, lon2)
        G.add_edge(row["ORIGIN"], row["DEST"], weight=distance)

    # Compute centrality metrics
    bc = nx.betweenness_centrality(G, normalized=True, weight=None)
    cc = nx.closeness_centrality(G, distance="weight")

    # Convert to Spark DataFrames
    bc_df = spark.createDataFrame([
        Row(ORIGIN=node, betweenness_unweighted=float(score)) 
        for node, score in bc.items()
    ])
    cc_df = spark.createDataFrame([
        Row(ORIGIN=node, closeness=float(score)) 
        for node, score in cc.items()
    ])

    return bc_df.join(cc_df, on="ORIGIN")


### 4.5 Weighted Betweenness Centrality

Uses flight frequency as edge weights (more flights = lower cost path).

In [0]:
def get_weighted_centrality(flight_df):
    """
    Calculate weighted betweenness centrality using flight frequency.
    
    Routes with more flights are treated as "shorter" paths (lower cost),
    so airports that connect high-traffic routes will have higher centrality.
    
    Args:
        flight_df: Flight DataFrame with ORIGIN, DEST columns
        
    Returns:
        DataFrame with ORIGIN and betweenness columns
    """
    rows = flight_df.groupBy("ORIGIN", "DEST").agg(F.count("*").alias("num_flights")).collect()

    G = nx.DiGraph()
    for r in rows:
        # Convert flight frequency to cost (more flights = lower cost)
        cost = 1.0 / r["num_flights"]
        G.add_edge(r["ORIGIN"], r["DEST"], weight=cost)

    bc = nx.betweenness_centrality(G, weight="weight", normalized=True)

    return spark.createDataFrame([
        Row(ORIGIN=node, betweenness=float(score))
        for node, score in bc.items()
    ])



### 4.6 Historical Delay Features

Airport-level delay statistics capture historical patterns that may predict future delays.

In [0]:
def average_delay_at_origin(flight_df):
    """
    Calculate average departure delay at each origin airport.
    
    Captures airport-level delay tendencies that may affect future flights.
    """
    return flight_df.groupBy("ORIGIN").agg(F.avg("DEP_DELAY").alias("avg_origin_dep_delay"))


def average_delay_at_dest(flight_df):
    """
    Calculate average arrival delay at each destination airport.
    
    Airports with historically high arrival delays may indicate congestion issues.
    """
    return flight_df.groupBy("DEST").agg(F.avg("ARR_DELAY").alias("avg_dest_arr_delay"))

## 5. Edge (Route) Features

Route-level features capture characteristics of specific origin-destination pairs, including traffic volume, historical delays, and hourly patterns.

In [0]:
def average_daily_flights_on_route(flight_df, dataset_n_days):
    """
    Calculate average daily flight count for each route (ORIGIN → DEST).
    
    High-traffic routes may have different delay patterns than low-traffic routes.
    """
    return flight_df.groupBy("ORIGIN", "DEST").agg(
        (F.count("*") / F.lit(dataset_n_days)).alias("avg_daily_route_flights")
    )


def average_delay_on_route(flight_df):
    """
    Calculate average departure delay for each route.
    
    Some routes may have consistently higher delays due to scheduling,
    weather patterns, or operational factors.
    """
    return flight_df.groupBy("ORIGIN", "DEST").agg(
        F.avg("DEP_DELAY").alias("avg_route_delay")
    )


def average_flights_at_hour(flight_df, dataset_n_days):
    """
    Calculate average flight count at each hour for each route.
    
    Captures hourly traffic patterns - rush hour flights may experience
    more delays than off-peak flights.
    """
    return flight_df.groupBy("ORIGIN", "DEST", "hour").agg(
        (F.count("*") / F.lit(dataset_n_days)).alias("avg_hourly_flights")
    )

## 6. Feature Pipeline

The feature pipeline orchestrates the computation and joining of all graph features.

In [0]:
# -----------------------------------------------------------------------------
# Feature Function Registry
# -----------------------------------------------------------------------------
# Organize feature functions by their join key for systematic computation

# Origin airport features (join on ORIGIN)
VERTEX_FUNCTIONS_ORIGIN = [
    page_rank,
    out_degree,
    in_degree,
    weighted_in_out_degree,
    get_runways,
    get_unweighted_centralities,
    get_weighted_centrality,
    average_delay_at_origin
]

# Destination airport features (join on DEST)
VERTEX_FUNCTIONS_DEST = [
    average_delay_at_dest
]

# Route features (join on ORIGIN, DEST)
EDGE_FUNCTIONS = [
    average_daily_flights_on_route,
    average_delay_on_route
]

# Hourly route features (join on ORIGIN, DEST, hour)
EDGE_HOURLY_FUNCTIONS = [
    average_flights_at_hour
]

In [0]:
def form_feature_df(functions, train_df, join_cols, dataset_n_days):
    """
    Compute and combine features from a list of feature functions.
    
    Args:
        functions: List of feature functions to apply
        train_df: Training DataFrame
        join_cols: Columns to join on
        dataset_n_days: Number of days in dataset for normalization
        
    Returns:
        DataFrame with all computed features joined together
    """
    assert len(functions) > 0, "At least one feature function required"
    
    # Compute first feature (handle functions with/without dataset_n_days param)
    try:
        df = functions[0](train_df, dataset_n_days)
    except TypeError:
        df = functions[0](train_df)
    
    # Join remaining features
    for func in functions[1:]:
        try:
            df = df.join(func(train_df, dataset_n_days), on=join_cols)
        except TypeError:
            df = df.join(func(train_df), on=join_cols)

    return df


def join_features(input_df, origin_df, dest_df, edge_df, edge_hour_df):
    """
    Join all computed features to the input flight DataFrame.
    
    Args:
        input_df: Base flight DataFrame
        origin_df: Origin airport features
        dest_df: Destination airport features  
        edge_df: Route features
        edge_hour_df: Hourly route features
        
    Returns:
        DataFrame with all features joined
    """
    return (
        input_df
        .join(origin_df, on="ORIGIN")
        .join(dest_df, on="DEST")
        .join(edge_df, on=["ORIGIN", "DEST"])
        .join(edge_hour_df, on=["ORIGIN", "DEST", "hour"])
    )

In [0]:
def add_features(dataset_dir):
    """
    Main pipeline function to compute and save graph features for a dataset.
    
    Steps:
    1. Load train/validation/test splits
    2. Compute all vertex and edge features from training data
    3. Join features to all splits (train uses same features for val/test)
    4. Save enriched datasets to parquet
    
    Args:
        dataset_dir: Name of dataset directory (e.g., '5_year_custom_joined')
    """
    print(f"Starting run for {dataset_dir}")
    dataset_path = f"{CHECKPOINT_PATH}/{dataset_dir}"
    graph_dir = "graph_feature_splits"
    dbutils.fs.mkdirs(f"{dataset_path}/{graph_dir}")

    # Step 1: Load data
    print("...Ingesting Data")
    train_df, validation_df, test_df = ingest_data(
        f"{dataset_path}/feature_eng_ph3/training_splits"
    )

    # Step 2: Compute features (using training data only to prevent leakage)
    print("...Computing Features")
    dataset_n_days = get_dataset_days(train_df)

    origin_df = form_feature_df(
        VERTEX_FUNCTIONS_ORIGIN, train_df, ["ORIGIN"], dataset_n_days
    )
    dest_df = form_feature_df(
        VERTEX_FUNCTIONS_DEST, train_df, ["DEST"], dataset_n_days
    )
    edge_df = form_feature_df(
        EDGE_FUNCTIONS, train_df, ["ORIGIN", "DEST"], dataset_n_days
    )
    edge_hour_df = form_feature_df(
        EDGE_HOURLY_FUNCTIONS, train_df, ["ORIGIN", "DEST", "hour"], dataset_n_days
    )

    # Step 3: Join features to all splits
    print("...Joining Features")
    train_df_joined = join_features(train_df, origin_df, dest_df, edge_df, edge_hour_df)
    validation_df_joined = join_features(validation_df, origin_df, dest_df, edge_df, edge_hour_df)
    test_df_joined = join_features(test_df, origin_df, dest_df, edge_df, edge_hour_df)

    # Step 4: Save to parquet
    print("...Writing Features")
    train_df_joined.write.parquet(f"{dataset_path}/{graph_dir}/train", mode="overwrite")
    validation_df_joined.write.parquet(f"{dataset_path}/{graph_dir}/validation", mode="overwrite")
    test_df_joined.write.parquet(f"{dataset_path}/{graph_dir}/test", mode="overwrite")
    
    print(f"✓ Completed {dataset_dir}")

## 7. Execution

Run the feature pipeline on all dataset sizes (3-month, 1-year, 5-year).

In [0]:
# Process all datasets
for dataset_dir in DATASET_DIRS:
    start_time = time.time()
    add_features(dataset_dir)
    elapsed = time.time() - start_time
    print(f"  → Completed in {elapsed/60.0:.2f} minutes\n")

In [0]:
# Verify output directories
display(dbutils.fs.ls(CHECKPOINT_PATH))

## 8. Summary

This notebook computes graph-based features that capture the network structure of the US flight system. These features provide the model with information about airport importance, connectivity, and historical patterns that raw flight data alone cannot capture.

### Features Generated

| Feature | Type | Description |
|---------|------|-------------|
| `page_rank` | Vertex | Airport importance based on incoming connections |
| `out_degree` | Vertex | Number of unique destinations |
| `in_degree` | Vertex | Number of unique origins |
| `weighted_in_degree` | Vertex | Average daily incoming flights |
| `weighted_out_degree` | Vertex | Average daily outgoing flights |
| `N_RUNWAYS` | Vertex | Physical runway count |
| `betweenness_unweighted` | Vertex | Centrality based on shortest paths |
| `betweenness` | Vertex | Centrality weighted by flight frequency |
| `closeness` | Vertex | Geographic closeness centrality |
| `avg_origin_dep_delay` | Vertex | Historical departure delay at origin |
| `avg_dest_arr_delay` | Vertex | Historical arrival delay at destination |
| `avg_daily_route_flights` | Edge | Average daily flights on route |
| `avg_route_delay` | Edge | Historical delay for route |
| `avg_hourly_flights` | Edge | Average flights at specific hour |

### Key Design Decisions

1. **Training Data Only**: All features are computed from training data to prevent data leakage
2. **Normalization**: Daily averages account for different dataset sizes
3. **Multiple Centrality Metrics**: Both weighted and unweighted versions capture different aspects of importance
4. **Hierarchical Features**: Vertex (airport) and edge (route) features at different granularities
